### Dataset

dataset was obtained in GDELT Project, specifically from the gdelt-bq project, covid19 dataset, and onlinenews table in Google BigQuery. 
The GDELT Project is a comprehensive initiative that monitors the world's news media, coding and analyzing news events globally.
The gdelt-bq.covid19.onlinenews table focuses on online news articles related to COVID-19. 
This dataset is widely used for tracking and analyzing media coverage and trends related to the COVID-19 pandemic.

link: https://console.cloud.google.com/bigquery?p=gdelt-bq&d=covid19&t=onlinenews&page=table&pli=1&project=aled7-297920&ws=!1m5!1m4!4m3!1sgdelt-bq!2scovid19!3sonlinenews!1m9!1m8!1m3!1saled7-297920!2sbquxjob_36e9fcb1_18fea093775!3sUS!14m3!1saled7-297920!2sbquxjob_5a2fe995_18fea190b75!3sUS

my filtering:
- 10 articles per day
- from '2020-01-01' AND '2022-12-31'
- topic covid-19 vaccin
- deleted duplicates



In [ ]:
# WITH RankedArticles AS (
#   SELECT *,
#          ROW_NUMBER() OVER (PARTITION BY DATE(DateTime), LOWER(Title) ORDER BY DateTime) AS title_rn
#   FROM `gdelt-bq.covid19.onlinenews`
#   WHERE
#     (LOWER(Topic) LIKE '%covid-19 vaccin%' OR LOWER(Title) LIKE '%covid-19 vaccin%')
#     AND DATE(DateTime) BETWEEN '2020-01-01' AND '2022-12-31'
# ),
# FilteredArticles AS (
#   SELECT *,
#          ROW_NUMBER() OVER (PARTITION BY DATE(DateTime) ORDER BY DateTime) AS rn
#   FROM RankedArticles
#   WHERE title_rn = 1
# )
# SELECT *
# FROM FilteredArticles
# WHERE rn <= 10

### 1. Import Necessary Libraries

In [2]:
!pip install pycountry


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.0 MB/s eta 0:00:0000:0100:01


In [3]:
# Import Necessary Libraries
import numpy as np
import pandas as pd
import re
# import matplotlib.pyplot as plt
# import seaborn as sns
from textblob import TextBlob
# from wordcloud import WordCloud, STOPWORDS
# from geotext import GeoText
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
import pycountry  # to map country abreviation to full name

import warnings
warnings.filterwarnings('ignore')

### 2. Load the Dataset

In [4]:
# df_covid =  pd.read_csv("/kaggle/input/qqqqqq/df_covid (2).csv")

In [ ]:
df = pd.read_csv("/kaggle/input/bigquery-covid-vacc/bigquery.csv")

print(df.columns)
print(df.shape)
df.info()

df.head()

### 3. Data Preprocessing
Select only interested columns, remove NaN, Clean the text data by removing duplicates and formatting the date.

In [ ]:
df.dropna()

In [ ]:
# date column with only YY-mm-dd
df['date'] = pd.to_datetime(df['DateTime']).dt.date

In [ ]:
# Function to extract media and full country name from URL
def extract_media_country_full_name(url):
    # Extract domain name
    media = re.findall(r'://(www\.)?([^/]+)/', url)
    media = media[0][1] if media else ''

    # Extract country code if present in the domain suffix
    country_code = re.findall(r'\.([a-z]{2})(/|$)', url)
    country_code = country_code[0][0] if country_code else ''
    
    # Convert country code to full name using pycountry
    country = ''
    if country_code:
        try:
            country = pycountry.countries.get(alpha_2=country_code.upper()).name
        except AttributeError:
            country = ''
    elif '.com' in url:
        country = 'Global'

    return media, country

# Apply the function to the URL column
df[['media', 'country']] = df['URL'].apply(lambda x: pd.Series(extract_media_country_full_name(x)))


### 4. Add companies (sp500 health care sector)

In [ ]:
def fetch_sp500_health_care_companies():
    """Retrieves a list of S&P 500 companies from Wikipedia"""
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(url)[0]
    health_care_companies = sp500_table[sp500_table['GICS Sector'] == 'Health Care']

    return health_care_companies[['Symbol', 'Security']]

# Fetch the list of S&P 500 companies
sp500_companies_df = fetch_sp500_health_care_companies()
sp500_symbols_list= sp500_companies_df['Symbol'].tolist()
sp500_companies_list = sp500_companies_df['Security'].tolist()
company_to_symbol = dict(zip(sp500_companies_list, sp500_symbols_list))

def extract_companies(text):
    """Extracts mentioned S&P 500 companies and symbols from text"""
    mentioned_companies = [entry for entry in sp500_companies_list if re.search(r'\b' + re.escape(entry) + r'\b', text, re.IGNORECASE)]
    return ', '.join(mentioned_companies) if mentioned_companies else ''

# Apply the function to both Title and Context columns
df['Security'] = df.apply(lambda row: extract_companies(row['Title'] + ' ' + row['Context']), axis=1)
df['Symbol'] = df['Security'].map(company_to_symbol)
df['Symbol'].fillna('', inplace=True)



In [ ]:
# print('number of articles with companies: ', df['Security'].apply(lambda x: x.strip() != '').sum())
# print('articles with companies identified: ', df[df['Security'].apply(lambda x: x.strip() != '')])

### 5. Filter columns and save

In [ ]:
df_covid = df[['Topic', 'Title', 'Context','date','media','country', 'Security','Symbol' ]]
df_covid

In [ ]:
# Save the modified DataFrame to a new CSV file
output_file_path = '/kaggle/working/df_covid.csv'
df_covid.to_csv(output_file_path, index=False)

output_file_path

### 6. Sentiment Analysis using TextBlob
Analyze the sentiment of each tweet using TextBlob. 
- polarity: emotional tone of the text (positive, negative, or neutral).
- subjectivity: if text is more opinion-based or fact-based.

In [ ]:
# Sentiment Analysis using TextBlob
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    return sentiment

# def get_subjectivity(text):
#     analysis = TextBlob(text)
#     return analysis.sentiment.subjectivity

# Apply the sentiment function to the Title and Context columns
df_covid['title_sentiment'] = df_covid['Title'].apply(get_sentiment)
df_covid['context_sentiment'] = df_covid['Context'].apply(get_sentiment)
df_covid['sentiment'] = df_covid[['title_sentiment', 'context_sentiment']].mean(axis=1)

# df_covid['title_subjectivity'] = df_covid['Title'].apply(get_subjectivity)
# df_covid['context_subjectivity'] = df_covid['Context'].apply(get_subjectivity)
# df_covid['combined_subjectivity'] = df_covid[['title_subjectivity', 'context_subjectivity']].mean(axis=1)


In [ ]:
# Calculate the average sentiment for each day
mean_sentiment_per_day = df_covid.groupby('date')['sentiment'].mean().reset_index()
mean_sentiment_per_day.rename(columns={'sentiment': 'day_sentiment'}, inplace=True)

# Merge to dataframe
df_covid = df_covid.merge(mean_sentiment_per_day, on='date', how='left')


In [ ]:
def categorize_sentiment(score):
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

df_covid['sentiment_cat'] = df_covid['sentiment'].apply(categorize_sentiment)
df_covid['day_sent_cat'] = df_covid['day_sentiment'].apply(categorize_sentiment)


### 7. Add day before and day after

In [ ]:
# Add daybefore and dayafter columns

df_covid['daybefore'] = df_covid['date'] - pd.Timedelta(days=1)
df_covid['dayafter'] = df_covid['date'] + pd.Timedelta(days=1)


In [ ]:
# Add the average sentiment for daybefore and dayafter
day_before_sent_cat = mean_sentiment_per_day[['date', 'day_sentiment']].copy()
day_before_sent_cat['date'] = day_before_sent_cat['date'] + pd.Timedelta(days=1)
day_before_sent_cat.rename(columns={'day_sentiment': 'dayBefore_sent_cat'}, inplace=True)

day_after_sent_cat = mean_sentiment_per_day[['date', 'day_sentiment']].copy()
day_after_sent_cat['date'] = day_after_sent_cat['date'] - pd.Timedelta(days=1)
day_after_sent_cat.rename(columns={'day_sentiment': 'dayAfter_sent_cat'}, inplace=True)

# Merge the day_before and day_after sentiment categories back to the original dataframe
df_covid = df_covid.merge(day_before_sent_cat, on='date', how='left')
df_covid = df_covid.merge(day_after_sent_cat, on='date', how='left')


### Remove

In [ ]:
# columns_to_remove = ['dayAfter_sent_cat']  # Replace with actual column names to be removed
# df_covid.drop(columns=columns_to_remove, inplace=True)


In [8]:
# df_covid['Symbol'].fillna('', inplace=True)
# df_covid['Security'].fillna('', inplace=True)
# df_covid['date'] = pd.to_datetime(df_covid['date']).dt.date

In [9]:
df_covid[1:20]

,Topic,Title,Context,date,media,country,Security,Symbol,title_sentiment,context_sentiment,sentiment,day_sentiment,sentiment_cat,day_sent_cat,daybefore,dayafter,dayBefore_sent_cat,dayAfter_sent_cat
1,Cases,"Faster route to Covid-19 vaccine possible, say...",KUALA LUMPUR (Bernama): With the number of cas...,2020-02-12,thestar.com.my,Malaysia,,,0.0,-0.162500,-0.081250,-0.008153,negative,negative,2020-02-11,2020-02-13,NaN,negative
2,Masks,San Diego lab discovers COVID-19 vaccine in 3 ...,"As the days go by, Inovio Pharmaceuticals is g...",2020-02-12,cbs8.com,Global,,,0.0,-0.200000,-0.100000,-0.008153,negative,negative,2020-02-11,2020-02-13,NaN,negative
3,Masks,San Diego lab discovers COVID-19 vaccine in 3 ...,"As the days go by, Inovio Pharmaceuticals is g...",2020-02-12,cbs8.com,Global,,,0.0,-0.200000,-0.100000,-0.008153,negative,negative,2020-02-11,2020-02-13,NaN,negative
4,Testing,J&J strengthens R&D into COVID-19 vaccine with...,The collaborative partnership with BARDA build...,2020-02-12,pharmaphorum.com,Global,,,0.0,0.115000,0.057500,-0.008153,positive,negative,2020-02-11,2020-02-13,NaN,negative
5,Testing,J&J strengthens R&D into COVID-19 vaccine with...,The collaborative partnership with BARDA build...,2020-02-12,pharmaphorum.com,Global,,,0.0,0.115000,0.057500,-0.008153,positive,negative,2020-02-11,2020-02-13,NaN,negative
6,Cases,Coronavirus outbreak: COVID-19 vaccine candida...,The total death toll in China climbed to 1113....,2020-02-12,theage.com.au,Australia,,,0.0,0.182273,0.091136,-0.008153,positive,negative,2020-02-11,2020-02-13,NaN,negative
7,Cases,Coronavirus outbreak: COVID-19 vaccine candida...,The total death toll in China climbed to 1113....,2020-02-12,theage.com.au,Australia,,,0.0,0.182273,0.091136,-0.008153,positive,negative,2020-02-11,2020-02-13,NaN,negative
8,Cases,Coronavirus outbreak: COVID-19 vaccine candida...,The total death toll in China climbed to 1113....,2020-02-13,watoday.com.au,Australia,,,0.0,0.182273,0.091136,-0.004432,positive,negative,2020-02-12,2020-02-14,negative,positive
9,Cases,Coronavirus outbreak: COVID-19 vaccine candida...,The total death toll in China climbed to 1113....,2020-02-13,watoday.com.au,Australia,,,0.0,0.182273,0.091136,-0.004432,positive,negative,2020-02-12,2020-02-14,negative,positive
10,Cases,Coronavirus outbreak: COVID-19 vaccine candida...,The total death toll in China climbed to 1113....,2020-02-13,watoday.com.au,Australia,,,0.0,0.182273,0.091136,-0.004432,positive,negative,2020-02-12,2020-02-14,negative,positive
